First we import libraries required that we'll be using

In [47]:
import numpy as np
import pandas as pd
import serial as sr
import sys

We'll also import our variable files `computer` which stores all USB ports for our respective computers

In [48]:
sys.path.append("../python")

In [49]:
from computers import *

We want to create a data frame for which we'll store our angle data

In [50]:
coordinates_frame = pd.DataFrame(columns=['x', 'y', 'z', 'angle_1', 'angle_2'])

We define our pins to be used on the `Arduino Mega 2560`. As a reminder, our pins are defined as

```c
LIDAR_POWER_ENABLE_PIN1 9
LIDAR_POWER_ENABLE_PIN2 10
```

In [51]:
pin_1 = True # Pin 9
pin_2 = False # Pin 10

Our loop iterations follow the following scheme:

```c
bool alternate = false;

  if(alternate){
    disableLidar(LIDAR_POWER_ENABLE_PIN1);
    enableLidar(LIDAR_POWER_ENABLE_PIN2);
    alternate = false;
  }
  else{
    enableLidar(LIDAR_POWER_ENABLE_PIN1);
    disableLidar(LIDAR_POWER_ENABLE_PIN2);
    alternate = true;
  }
```

Meaning in our first iteration:
```
LIDAR_POWER_ENABLE_PIN1 enabled
LIDAR_POWER_ENABLE_PIN2 disabled
```

and our second iteration:
```
LIDAR_POWER_ENABLE_PIN1 disabled
LIDAR_POWER_ENABLE_PIN2 enabled
```

and so forth...

We now define a function `get_x_coordinate` which retrieves our clean `x` distance

In [52]:
def get_x_coordinate(data: str) -> np.int64:
    x_coordinate = re.findall(r'\b\d+\b', data)
    if x_coordinate:
        return np.int64(x_coordinate[0])
    else:
        return np.int64(-1)

Deciphering the funciton, the function `get_x_coordinate` functions as:
1. Find all values matching the regex expression. `\b` find the beginning of a word character. `\d` finds the beginning of a digit character and matches digits _0-9_. \b is another word boundary anchor, ensuring digits are not part of a longer word.
2. If a match is found, x_coordinate will return the match in a list. We only expect to find one element, thus we will return the `0`th index.
3. If not found, we ensure this by returning -1

We define another function to process our coordinate data

In [53]:
def save_coordinate(x: int, y: int, z: int) -> np.array:
    # Perform math to get accurate coordinates
    return np.around(np.array([x, y, z]), 3)

We define a function to print coordinates

In [54]:
def print_coordinate(cords: np.array):
    print(f'X: {cords[0]}, Y: {cords[1]}, Z: {cords[2]}')

We now create a serial connection

In [ ]:
ser = sr.Serial(port=PABLOS_COMPUTER, baudrate=9600)  # Adjust the serial port and baud rate
print(f'Connection to port {ser} is established ...')

We now run an infinite loop until we decide to shut down the system

In [ ]:
while True:
    # Read data from the serial port
    data = sr.Serial.readline().decode('utf-8').strip()
    
    # Get lidar points
    x = get_x_coordinate(data)
    y = 0
    z = -1

    coordinates = save_coordinate(x, y, z)
    angle_df.loc[len(angle_df.index)] = coordinates
    print_coordinate(coordinates)

    # LIDAR logic
    if pin_1:
        pin_1 = False
        pin_2 = True
    else:
        pin_1 = True
        pin_2 = False

We now save our coordinates into an excel file

In [ ]:
coordinates_frame.to_csv("../../data/coordinates.csv")

## MOCK TEST

In [56]:
import random

In [57]:
def get_fake_x_coordinate(data: str) -> np.int64:
    x_coordinate = re.findall(r'\b\d+\b', data)
    if x_coordinate:
        return np.int64(x_coordinate[0])
    else:
        return np.int64(-1)

In [58]:
def save_fake_coordinate(x: int, y: int, z: int) -> np.array:
    # Perform math to get accurate coordinates
    return np.around(np.array([x, y, z]), 3)

In [59]:
def generate_fake_coordiantes(num):
    x = np.random.uniform(0, 2000, num)
    y = np.random.uniform(0, 2000, num)
    z = np.random.uniform(0, 2000, num)
    return np.array([x, y, z])

In [68]:
def generate_fake_measurements(num):
    measurement = []
    for _ in range(num):
        x = np.random.randint(0, 2000)
        measurement.append(f'{x} cm')
    return np.array(measurement)

In [82]:
fake_serial_data = generate_fake_measurements(10)

In [83]:
fake_coordinates = generate_fake_coordiantes(10)

In [90]:
x_batch, y_batch, z_batch = fake_coordinates

In [96]:
fake_df = pd.DataFrame(columns=["x", "y", "z"])

In [98]:
for x, y, z in zip(x_batch, y_batch, z_batch):
    fake_df.loc[len(fake_df.index)] = np.around(np.array([x, y, z]), 3)
    print_coordinate([x, y, z])

X: 566.8146541930665, Y: 689.7454125418736, Z: 45.67971917940827
X: 1794.6502595244383, Y: 5.624410445841166, Z: 454.15590680390784
X: 873.5443082629888, Y: 1354.6332025104946, Z: 284.5456792332404
X: 1548.7155785383673, Y: 694.85701620343, Z: 1829.8478650906816
X: 544.8396104008535, Y: 55.69524997970032, Z: 198.39910477069145
X: 1795.135932770606, Y: 228.67828513388355, Z: 1754.8636502853155
X: 203.44432247837508, Y: 1079.1046480231275, Z: 99.85783435182616
X: 62.106911932354066, Y: 1833.8102224150232, Z: 1711.31393233021
X: 64.29281919448692, Y: 1923.46078770615, Z: 288.3404994666021
X: 11.224267504238217, Y: 993.0111215114446, Z: 1082.4264791487897


In [99]:
fake_df.head(10)

,x,y,z
0,566.815,689.745,45.680
1,1794.650,5.624,454.156
2,873.544,1354.633,284.546
3,1548.716,694.857,1829.848
4,544.840,55.695,198.399
5,1795.136,228.678,1754.864
6,203.444,1079.105,99.858
7,62.107,1833.810,1711.314
8,64.293,1923.461,288.340
9,11.224,993.011,1082.426
